# Chapter 8 - 직교 행렬과 QR 분해: 선형대수학의 핵심 분해법 1

In [1]:
import numpy as np
import pandas as pd
import scipy.signal


import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg') # display figures in vector format
plt.rcParams.update({'font.size':14}) # set global font size

# 폰트 설정
%config InlineBackend.figure_format = 'retina'
plt.rcParams["axes.unicode_minus"] = False

plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

## 8-1. 난수 행렬 Q를 생성하고, Q.T, Q^-1을 예산하는 코드 작성하여 구현

In [2]:
# 난수 직교 행렬 생성 → 난수 행렬을 생성한 후, QR분해를 통해 직교 행렬 Q를 return

def random_orthogonal_matrix(n):

    # n * n 크기의 난수 행렬 (정방 행렬)
    matrix = np.random.randn(n, n)

    # QR 분해 → 직교 행렬 Q 생성
    q, r = np.linalg.qr(matrix)

    return q

def compare_matrix(n):

    q = random_orthogonal_matrix(n)

    print("q.T @ q :", "\n", q.T @ q, "\n")
    print("q @ q.T :", "\n", q @ q.T, "\n")
    print("np.linalg.inv(q) @ q :", "\n", np.linalg.inv(q) @ q, "\n")
    print("q @ np.linalg.inv(q) :", "\n", q @ np.linalg.inv(q), "\n")
    print("단위 행렬: ", "\n", np.eye(n), "\n")

    return np.allclose(q.T @ q, np.eye(n))

In [3]:
compare_matrix(3)

q.T @ q : 
 [[ 1.00000000e+00 -9.71445147e-17 -5.55111512e-17]
 [-9.71445147e-17  1.00000000e+00  7.63278329e-17]
 [-5.55111512e-17  7.63278329e-17  1.00000000e+00]] 

q @ q.T : 
 [[ 1.00000000e+00 -3.33066907e-16  1.66533454e-16]
 [-3.33066907e-16  1.00000000e+00 -1.11022302e-16]
 [ 1.66533454e-16 -1.11022302e-16  1.00000000e+00]] 

np.linalg.inv(q) @ q : 
 [[ 1.00000000e+00  7.63278329e-17 -5.55111512e-17]
 [-5.55111512e-17  1.00000000e+00 -3.46944695e-18]
 [ 0.00000000e+00  7.63278329e-17  1.00000000e+00]] 

q @ np.linalg.inv(q) : 
 [[ 1.00000000e+00  1.11022302e-16  5.55111512e-17]
 [-5.55111512e-17  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]] 

단위 행렬:  
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] 



True

## 8-2. 그람-슈미트 과정 구현
- 4 * 4 난수 행렬 사용
- 결과를 `np.linalg.qr`의 Q와 대죠하여 확인

In [4]:
# V = np.random.randint(1, 10, size=(4, 4))
# V

In [5]:
# for k in np.arange(1, V.shape[1]):
#     u_1 = V[:, 0]
#     v_k = V[:, k] * np.dot(u_1, )

In [6]:
def gram_schmidt(m, n):
    
    global V
    V = np.random.randint(1, 10, size=(m, n))
    Q = np.zeros((m, n))

    for i in range(n):
        
        Q[:,i] = V[:,i]
        v = V[:,i]

        for j in range(i):
            q = Q[:,j]
            Q[:,i]=Q[:,i]-np.dot(v,q)/np.dot(q,q)*q
        
        # normalize
        Q[:,i] = Q[:,i] / np.linalg.norm(Q[:,i])

    return Q

In [8]:
gram_schmidt(4, 4)

array([[ 0.09622504,  0.15097027,  0.9258201 ,  0.33287514],
       [ 0.48112522,  0.41516825,  0.15430335, -0.75653442],
       [ 0.8660254 , -0.33968311, -0.15430335,  0.33287514],
       [ 0.09622504,  0.83033649, -0.3086067 ,  0.45392065]])

In [11]:
Q, R = np.linalg.qr(V)

In [13]:
gram_schmidt(4, 4) - Q

array([[ 0.70543274,  0.01145586, -1.36218904,  0.31441448],
       [ 1.09033292,  0.27565384, -0.37145732,  0.00670386],
       [ 1.27216387, -0.62464871,  1.01205217, -0.19829017],
       [ 0.40082889,  1.76834825,  0.47198743, -0.42828542]])

In [14]:
gram_schmidt(4, 4) + Q

array([[ 0.49538293,  0.19212238,  0.41650308,  0.85525777],
       [-0.22757895, -0.60822038,  0.90813091, -0.18191352],
       [-0.10538657,  0.04060235, -0.05420944, -0.23454047],
       [-0.01170962,  0.03889823,  0.09428459,  0.1801201 ]])

## 8-3. 거의 직교에 가깝지만 직교는 아닌 행렬에 QR 분해 적용
1. 6 * 6 난수 행렬의 QR 분해로부터 U라는 직교 행렬 만든다
2. U의 QR 분해 계산하고 R = 1임을 확인한다 (그 이유도 이해해야 한다)
3. U의 각 norm을 수정한다
    - 1 ~ 6열의 norm 10 ~ 15로 수정 (첫 번째 열의 norm 10, 두 번재 열의 norm 11,...)
    - 변조된 U행렬을 QR 분해하여 그 R의 대각선 원소가 10 ~ 15인 대각행렬인지 확인한다
    - 이 행렬의 Q.T @ Q는?
4. 원소 u_1,4 = 0으로 설정하여 U의 직교성을 깨뜨리면, R은 어떻게 되고 그 이유는 뭘까?

In [67]:
# 6 * 6 난수 행렬의 QR 분해로부터 U라는 직교 행렬을 만든다

# U = np.random.randint(1, 10, size=(6, 6))
U = np.linalg.qr(np.random.randn(6, 6))[0]
Q, R = np.linalg.qr(U)

# 1. U의 QR 분해를 계산하고 R=I 임을 확인함 (True)
np.allclose(R, np.eye(6))

True

In [68]:
# 3. U의 각 열의 노름 수정
# - (10 ~ 15)로 수정
for i in range(len(U)):
    U[:, i] = U[:,i] * (10+i)
U

array([[-4.75947586,  1.58751153,  7.80203417,  6.35782973,  4.21725851,
         0.01392422],
       [ 4.13163856, -6.29699629,  6.24863111, -0.54686903, -1.77973979,
        -6.91487083],
       [-5.32445502, -7.10772154, -4.05177239, -1.50400148,  5.395219  ,
        -2.2788486 ],
       [ 1.14050538, -0.46938163,  4.45928886, -8.22999152,  5.94743012,
         7.73373326],
       [-5.33781937,  2.03886103,  2.59292331, -7.11087849, -7.13498746,
        -4.1106268 ],
       [-1.46084322, -4.89189799,  1.02835019,  2.7784099 , -7.72391726,
         9.76131824]])

In [69]:
# 변조된 U 행렬을 QR 분해해서 그 R의 대각선 원소가 10 ~ 15인 대각행렬인지 확인
Q, R = np.linalg.qr(U)

# R의 대각선 원소 : 10 ~ 15인 대각행렬
np.diag(R)

array([10., 11., 12., 13., 14., 15.])

In [70]:
# 이 행렬의 QTQ 확인 → 단위 행렬
np.allclose(Q.T @ Q, np.eye(6))

True

In [74]:
# 4. 원소 u_{1,4} = 0으로 설정하여 U의 직교성을 깨뜨린다. R은 어떻게 될까?
U[0, 3] = 0

Q, R = np.linalg.qr(U)
print(R, "\n")
print(np.diag(R))


[[ 1.00000000e+01  6.66133815e-16  1.77635684e-15  3.02599372e+00
   0.00000000e+00 -2.86229374e-16]
 [ 0.00000000e+00  1.10000000e+01 -1.77635684e-15 -9.17557093e-01
  -1.11022302e-16  2.66453526e-15]
 [ 0.00000000e+00  0.00000000e+00  1.20000000e+01 -4.13366707e+00
   0.00000000e+00  9.99200722e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00743361e+01
  -2.66147676e+00 -8.78745914e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.37446914e+01 -1.70157464e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.49999973e+01]] 

[10.         11.         12.         10.07433614 13.74469139 14.99999733]


## 8-6. 정방 직교 행렬의 모든 특이값 및 고유값 = 1,

## 8-7. QR 분해를 사용하여 최소 제곱법을 구현하는 방법을 이해하는데 도움이 되는 R 행렬

In [19]:
m, n = 10, 4
matrix = np.random.randint(1, 10, size=(m, n))
Q, R = np.linalg.qr(matrix)

In [20]:
np.round(R)

array([[-18., -18., -16., -16.],
       [  0.,  10.,   8.,   7.],
       [  0.,   0.,  -7.,  -8.],
       [  0.,   0.,   0.,  -4.]])

In [22]:
inverse = np.linalg.inv(R[:n, :])
tall_inverse = np.linalg.pinv(R)

In [23]:
inverse

array([[-0.05707301, -0.09885545,  0.01679211,  0.0210013 ],
       [ 0.        ,  0.09853449,  0.11434803, -0.05444963],
       [-0.        , -0.        , -0.14523156,  0.27275233],
       [-0.        , -0.        , -0.        , -0.23227681]])

In [25]:
np.allclose(tall_inverse, inverse)

True